# Técnica de reducción de la dimensionalidad

La reducción de la dimensionalidad es una técnica que se utiliza en la minería de datos para poder transformar datasets de alta dimensionalidad a unas que tengan una menor dimensionalidad. De esta forma se consiguen unas visualizaciones mucho más simples, y además, facilita mucho la búsqueda de patrones complejos, que a simple vista serían imposibles de detectar en los datos originales.

También pasa que al tener un montón de atributos, se pueden dar un montón de combinaciones diferentes por lo que para el modelo es mucho más complicado aprender y esto conlleva que el modelo sobreajuste demasiado. Justamente la principal función que cumplen las técnicas de reducción de la dimensionalidad es prevenir el sobreajuste.

## Lectura de datos

Primero importamos las librerías que necesitaremos durante el ejemplo.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine

En esta ocasión utilizaremos el dataset de wine, en el que tenemos diferentes atributos de vinos junto con el tipo de vino que pertenece cada uno.

In [ ]:
wines = load_wine()
data = pd.DataFrame(data = np.c_[wines['data'], wines['target']],
                     columns = wines['feature_names'] + ['target'])
data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0


Ahora dividiremos los atributos de la variable objetivo.

In [ ]:
X = data.iloc[:, 0:13]
y = data.iloc[:, 13]

Y también dividiremos las observaciones de los atributos en train y test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

En este notebook se mostrará cómo reducir la dimensionalidad del dataset con las técnicas de PCA y LDA, y para ello es muy recomendable escalar los datos.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_sc = scaler.transform(X)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

Aún así, primero vamos a crear un modelo de regresión logística con los datos únicamente escalados, para después poder comparar los resultados.

In [ ]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=42)
lg.fit(X_train_sc, y_train)

LogisticRegression(random_state=42)

Predecimos.

In [ ]:
y_pred = lg.predict(X_test_sc)
y_pred

array([0., 0., 2., 0., 1., 0., 1., 2., 1., 2., 0., 2., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 2., 2., 2., 1., 1., 1., 0., 0., 1., 2., 0.,
       0., 0., 2., 2., 1., 2., 0., 1., 1., 2., 2., 0., 1., 1., 2., 0., 1.,
       0., 0., 2., 2., 1., 0., 0., 1., 0., 2., 1., 0., 2., 0., 0., 0., 2.,
       0., 0., 0., 2., 1., 0., 2., 1., 0., 2., 1., 1., 0., 2., 0., 0., 1.,
       0., 0., 2., 1., 1., 1., 0., 1., 1., 1., 2., 2., 0., 1., 2., 2., 2.,
       1., 0., 1., 2., 2., 1., 2., 1., 1., 1., 0., 0., 2., 0., 2., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 2., 2., 1., 1., 1., 2., 2., 1., 0., 0.,
       1., 2., 2., 0., 1., 2., 2.])

En la matriz de confusión podemos ver que el modelo que hemos creado tiene una precisión muy alta.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[48,  0,  0],
       [ 3, 50,  4],
       [ 0,  0, 38]])

## Análisis de componentes principales (PCA)

Es una técnica que permite reducir la dimensionalidad del dataset minimizando la pérdida de información en el proceso. Pertenece a la familia de las técnicas no supervisadas de machine learning, y estas técnicas no necesitan variables objetivo. El principal problema de este tipo de técnicas suele ser que son difíciles de validar porque no tienen la variable objetivo para poder contrastar los resultados.

Ahora crearemos una instancia de la clase PCA que después utilizaremos para transformar los datos originales.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

Transformamos los datos.

In [ ]:
pca_fitted = pca.fit(X_train_sc)
X_train_pca = pca_fitted.transform(X_train_sc)
X_test_pca = pca_fitted.transform(X_test_sc)

Obtenemos los valores de los ratios de la varianza explicada para ver cuantas nuevas variables necesitaremos para poder explicar la información de los datos originales.

In [ ]:
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.39794788, 0.22572919, 0.12674483, 0.06712853, 0.03788473,
       0.03522223, 0.0262903 , 0.02372236, 0.01990599, 0.01577964,
       0.01206248, 0.0081107 , 0.00347113])

In [ ]:
cum_sum_eigenvalues = np.cumsum(exp_var_pca)

Podemos ver que con 8 componentes pasamos los 0.92, por lo que probaremos con las 8 primeras componentes principales y analizaremos los resultados. Aun así, la cantidad de varianza explicada que hay que superar es un parámetro que cada usuario determinará a su gusto. 

In [ ]:
import plotly.express as px

px.area(
    x=range(1, cum_sum_eigenvalues.shape[0] + 1),
    y=cum_sum_eigenvalues,
    labels={"x": "# Components", "y": "Explained Variance"},
    width=800, height=400
)

Como hemos comentado vamos a coger las primeras 8 componentes principales y vamos a ver como sería crear un modelo simple de regresión logística.

In [ ]:
pca = PCA(n_components = 8)

pca_fitted = pca.fit(X_train_sc)

X_train_pca = pca_fitted.transform(X_train_sc)
X_test_pca = pca_fitted.transform(X_test_sc)

Ahora veremos como sería crear un modelos de regresión logística utilizando el dataset que hemos obtenido con el PCA.

In [ ]:
lg = LogisticRegression(random_state=42)
lg.fit(X_train_pca, y_train)

LogisticRegression(random_state=42)

In [ ]:
y_pred = lg.predict(X_test_pca)
y_pred

array([0., 0., 2., 0., 1., 0., 1., 2., 1., 2., 0., 2., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 1., 2., 2., 2., 1., 1., 1., 0., 0., 1., 2., 0.,
       0., 0., 2., 2., 1., 2., 0., 1., 1., 0., 2., 0., 1., 1., 2., 0., 1.,
       0., 0., 2., 2., 1., 0., 0., 1., 0., 2., 1., 0., 2., 0., 0., 0., 2.,
       0., 0., 0., 2., 1., 0., 2., 1., 0., 2., 1., 1., 0., 2., 0., 0., 1.,
       0., 0., 2., 1., 1., 1., 0., 1., 1., 1., 2., 2., 0., 1., 2., 2., 2.,
       1., 0., 1., 2., 2., 1., 2., 1., 1., 1., 0., 0., 2., 0., 2., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 2., 1., 1., 1., 2., 2., 1., 0., 0.,
       1., 2., 2., 0., 1., 2., 2.])

Podemos apreciar en la matriz de confusión que el modelo que hemos creado con los datos transformados tiene la misma precisión que el modelo anterior, pero hay que tener en cuenta que esta vez hemos uilizado 5 variables menos, por lo que, en esta ocasión, se puede decir que hemos acertado utilizando el PCA.

In [ ]:
confusion_matrix(y_test, y_pred)

array([[48,  0,  0],
       [ 4, 51,  2],
       [ 0,  0, 38]])

## Análisis discriminante lineal

Es un algoritmo predictivo para clasificaciones multiclase, pero también se puede utilizar como técnica para la reducción de la dimensionalidad. En este caso nos encontramos ante un algoritmo supervisado, por lo que necesitaremos la variable objetivo.

Cuando el objetivo del PCA es encontrar los componentes con mayor varianza posible, el de LDA es maximizar la separación de clases entre los componentes.

Crearemos una instancia de la clase LDA para hacer el ejemplo de reducción de dimensionalidad.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()

Transformamos los datos escalados previamente.

In [ ]:
lda_fitted = lda.fit(X_train_sc, y_train)

X_train_lda = lda_fitted.transform(X_train_sc)
X_test_lda = lda_fitted.transform(X_test_sc)

Obtenemos los valores de los ratios de la varianza explicada para ver cuantas nuevas variables necesitaremos para poder explicar la información de los datos originales.

In [ ]:
exp_var_pca = lda.explained_variance_ratio_
exp_var_pca

array([0.70699454, 0.29300546])

In [ ]:
cum_sum_eigenvalues = np.cumsum(exp_var_pca)

Podemos ver que, como esperábamos, solo ha creado 2 componentes, ya que la variable objetivo tiene 3 clases diferentes. 
 
También vemos que con solo un componente no llega a los 0.71, pero si cogemos los 2 componentes llega al 1, por lo que escogeremos los dos componentes.

In [ ]:
import plotly.express as px

px.area(
    x=range(1, cum_sum_eigenvalues.shape[0] + 1),
    y=cum_sum_eigenvalues,
    labels={"x": "# Components", "y": "Explained Variance"},
    width=800, height=400
)

Con la transformación del dataset que hemos obtenido crearemos un modelo muy simple de regresión logística para ver su rendimiento.

In [ ]:
lg = LogisticRegression(random_state=42)

Entrenamos el modelo.

In [ ]:
lg.fit(X_train_lda, y_train)

LogisticRegression(random_state=42)

Hacemos las predicciones.

In [ ]:
y_pred = lg.predict(X_test_lda)
y_pred

array([0., 0., 2., 0., 1., 0., 1., 2., 1., 2., 0., 2., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 2., 2., 2., 2., 1., 1., 1., 0., 0., 1., 2., 0.,
       0., 0., 2., 2., 1., 2., 0., 1., 1., 2., 2., 0., 1., 1., 2., 0., 1.,
       0., 0., 2., 2., 1., 2., 0., 1., 0., 2., 1., 2., 2., 0., 0., 0., 2.,
       0., 0., 2., 2., 1., 0., 2., 1., 0., 2., 1., 2., 0., 2., 0., 0., 1.,
       0., 0., 2., 1., 1., 1., 0., 1., 1., 1., 2., 2., 0., 1., 2., 2., 2.,
       1., 0., 1., 2., 2., 1., 2., 2., 1., 1., 0., 0., 2., 0., 2., 0., 0.,
       1., 2., 0., 0., 0., 1., 0., 2., 2., 1., 1., 1., 2., 2., 1., 0., 0.,
       1., 2., 2., 0., 1., 2., 2.])

En la matriz de confusión podemos ver que los resultados que hemos obtenido diferentes, ya que el error cometido solo afecta a las clases 2 y 3 (la clase 1 se predice a la perfección). Además, en este caso solo tenemos dos componentes por lo que podemos aprovechar las ventajas que esto conlleva.

In [ ]:
confusion_matrix(y_test, y_pred)

array([[48,  0,  0],
       [ 0, 46, 11],
       [ 0,  0, 38]])